In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 50
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.2        0.         0.
  0.2        0.2        0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.2       ]
 [0.         0.         0.         0.         0.         0.25
  0.         0.         0.16666667 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.14285714 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.33333333]
 [0.2        0.         0.         0.         0.         0.
  0.2        0.         0.16666667 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.14285714 0.         0.         0.14285714
  0.14285714 0.14285714 0.14285714 0.         0.         0.
  0.125      0.         0.         0.         0.  

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[2, 4, 2, 2, 3, 6, 3, 4, 2, 5, 7, 6, 5, 7, 6, 4, 6, 3, 4, 3]
1
3


In [6]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split





X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


X, y = random_split(X_train, y_train, no_users, 1234)

In [7]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 3.0, 3.0])
W2 = np.array([-2.0, 2.0, 3.0, -3.0])
W3 = 2 * W1
W4 = 2  * W2
W = [W1, W2]
m = 200
n = 4

scaler = [1.0, -1.0]

noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
        data = X[count]
        data[:, 0:4] *= scaler[i]
        datapoints[count] = {
                'features': data,
                'degree': node_degree(count, G),
                'label': y[count],
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [9]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(9, 4, bias=False)
        self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        #output = self.fc3(x)
        return output

In [10]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [11]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        #new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        #vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(47.0235, grad_fn=<MseLossBackward0>) tensor([ 0.0152,  0.0092, -0.0655,  0.2267, -0.3228, -0.2686, -0.3301,  0.1126,
        -0.0394, -0.4880,  0.1115,  0.8335, -0.0887, -0.2546, -0.2907, -0.3002,
         0.4460, -2.2253, -0.4600,  0.4040,  0.0984,  0.1270, -0.0244,  0.0132,
        -0.0421, -0.1371, -0.4225, -0.1874,  0.3231, -0.8927,  0.4509, -0.4262,
        -0.1263, -0.3973,  0.0710, -0.7483, -1.5357, -1.7467, -3.6251, -1.2475]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(52.0178, grad_fn=<MseLossBackwar

        -4.0797e-01, -1.5751e+00, -1.7819e+00, -3.2785e+00, -1.5227e+00]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(52.6346, grad_fn=<MseLossBackward0>) tensor([ 0.0324,  0.0146, -0.0998,  0.3893, -0.3332, -0.2901, -0.3446, -0.0793,
         0.1571, -0.4194,  0.2125,  1.2375, -0.4658,  0.5467,  0.5942,  0.5274,
         0.1081, -1.8170, -0.4652,  0.4214,  0.0997, -0.0467,  0.1348,  0.2015,
         0.1201, -0.2051, -0.3993, -0.2504,  0.4929, -0.8825, -0.1061,  0.2059,
         0.4398,  0.2954, -0.0057, -0.1840, -1.6867, -1.4499, -4.0686, -1.3261]) tensor(

2 tensor(43.1426, grad_fn=<MseLossBackward0>) tensor([-0.0741,  0.0478, -0.0332,  0.2938, -0.4404, -0.3010, -0.4456,  0.0672,
         0.1188, -0.7313,  0.4216,  0.9954, -0.4385,  0.3495,  0.4329,  0.3372,
         0.2655, -2.1865, -0.4205,  0.3603,  0.3063, -0.0889,  0.0716,  0.1156,
         0.0881,  0.0463, -0.2909,  0.1254,  0.0229, -0.6192, -0.1412, -0.0496,
         0.2373,  0.0473,  0.4117, -0.6123, -2.0320, -1.6228, -3.5221, -1.3218]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
2 tensor(71.4659, grad_fn=<MseLossBackward0>) tensor([-0.1852,  0.1109,  0.2426, 

torch.Size([50, 1])
torch.Size([50, 1])
2 tensor(44.2423, grad_fn=<MseLossBackward0>) tensor([-1.5409e-01,  1.6859e-01, -5.2306e-04,  4.0915e-01, -4.0601e-01,
        -2.7275e-01, -4.2042e-01,  6.9891e-02,  1.7734e-02, -2.9643e-01,
         3.0144e-01,  4.2596e-01, -2.9568e-01,  1.3551e-01,  7.6462e-02,
         8.7127e-02,  5.5483e-01, -8.9083e-01, -4.6558e-01,  4.0750e-01,
        -2.2863e-02, -3.6804e-02,  6.0646e-02,  1.1635e-01,  3.0602e-02,
        -1.2093e-01, -9.2405e-02, -4.1741e-01,  5.5657e-01, -9.2449e-01,
        -3.9386e-01,  3.9742e-01,  6.4582e-01,  3.9613e-01,  3.0851e-01,
        -3.9311e-01, -2.3022e+00, -1.0920e+00, -2.6709e+00, -1.7015e+00]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -

5 tensor(47.0235, grad_fn=<MseLossBackward0>) tensor([ 0.0152,  0.0092, -0.0655,  0.2267, -0.3228, -0.2686, -0.3301,  0.1126,
        -0.0394, -0.4880,  0.1115,  0.8335, -0.0887, -0.2546, -0.2907, -0.3002,
         0.4460, -2.2253, -0.4600,  0.4040,  0.0984,  0.1270, -0.0244,  0.0132,
        -0.0421, -0.1371, -0.4225, -0.1874,  0.3231, -0.8927,  0.4509, -0.4262,
        -0.1263, -0.3973,  0.0710, -0.7483, -1.5357, -1.7467, -3.6251, -1.2475]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
5 tensor(52.0178, grad_fn=<MseLossBackward0>) tensor([-0.0244,  0.0097,  0.0729, 

...

         0.4297,  0.0318,  0.5621, -0.7090, -1.6400, -1.8189, -4.7509, -1.2210]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(51.4716, grad_fn=<MseLossBackward0>) tensor([-2.3031e-01,  1.8165e-01, -1.6926e-01,  4.4679e-01, -4.7382e-01,
        -2.3629e-01, -4.2067e-01,  2.7646e-03,  1.0064e-02, -2.3193e-01,
         2.5040e-01,  9.8199e-01, -1.8288e-01,  3.1149e-02, -2.7661e-01,
         2.0761e-01,  4.9711e-01, -1.0008e+00, -5.7320e-01,  4.9896e-01,
        -1.1614e-01,  1.4121e-01, -4.8902e-02, -4.1563e-02, -3.6008e-02,
        -1.9402e-01, -1.7994e-01, -

92 tensor(57.7881, grad_fn=<MseLossBackward0>) tensor([ 0.1180, -0.1608,  0.0573,  0.4532, -0.6506, -0.4613, -0.5968,  0.0656,
         0.0952, -0.1512, -0.2137,  1.0715,  0.0685, -0.6441, -1.3928, -0.7339,
         0.7034, -1.0632, -0.3117,  0.2163, -0.0877, -0.0223,  0.0224,  0.0337,
         0.0300, -0.1995, -0.2968,  0.1634, -0.1488, -0.8258,  0.1696, -0.4593,
         0.2278, -0.3143,  0.2370, -0.1074, -2.6811, -1.7609, -2.0528, -1.0219]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
92 tensor(48.4883, grad_fn=<MseLossBackward0>) tensor([-0.0187,  0.0578, -0.0621

torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(52.0178, grad_fn=<MseLossBackward0>) tensor([-0.0244,  0.0097,  0.0729,  0.2128, -0.2789, -0.2013, -0.2983,  0.1257,
         0.0774, -0.8785,  0.3474,  2.2168, -0.5825,  0.6776,  0.7346,  0.5819,
        -0.0248, -2.5566, -0.5707,  0.3934,  0.3354, -0.0720,  0.1533,  0.2485,
         0.1621, -0.2508, -0.4534, -0.5423,  0.4441, -0.3449, -0.2889, -0.0327,
         0.4297,  0.0318,  0.5621, -0.7090, -1.6400, -1.8189, -4.7509, -1.2210]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(51.4716, grad_fn=<MseLossBackw

torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(52.6346, grad_fn=<MseLossBackward0>) tensor([ 0.0324,  0.0146, -0.0998,  0.3893, -0.3332, -0.2901, -0.3446, -0.0793,
         0.1571, -0.4194,  0.2125,  1.2375, -0.4658,  0.5467,  0.5942,  0.5274,
         0.1081, -1.8170, -0.4652,  0.4214,  0.0997, -0.0467,  0.1348,  0.2015,
         0.1201, -0.2051, -0.3993, -0.2504,  0.4929, -0.8825, -0.1061,  0.2059,
         0.4398,  0.2954, -0.0057, -0.1840, -1.6867, -1.4499, -4.0686, -1.3261]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(55.9117, grad_fn=<MseLossBackw

99 tensor(45.2610, grad_fn=<MseLossBackward0>) tensor([ 0.1354, -0.0807, -0.0078,  0.3358, -0.2964, -0.2533, -0.3703,  0.1283,
         0.1407, -0.0480,  0.0632,  1.3674, -0.7643,  0.3101, -0.0305,  0.2660,
         1.2382, -0.1131, -0.3213,  0.3047, -0.0589, -0.0235,  0.0912,  0.0793,
         0.0380,  0.0618, -0.2147, -0.2281,  0.3903, -1.4909,  0.3990, -0.1413,
         0.0297, -0.3017, -0.0355, -0.3138, -2.1324, -1.5592, -2.4731, -1.6009]) tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(44.2423, grad_fn=<MseLossBackward0>) tensor([-1.5409e-01,  1.6859e-01,

In [12]:
parameters_to_vector(model.parameters())

tensor([-0.2042, -0.2578, -0.0603, -0.2144,  0.1683, -0.1041,  0.1239, -0.2471,
        -0.1132, -0.2566, -0.2238, -0.0980,  0.0091, -0.0050,  0.3077, -0.3188,
        -0.2549,  0.2537,  0.1664, -0.3067, -0.1337, -0.1716, -0.2813,  0.0414,
        -0.1889, -0.0123,  0.2228, -0.2023, -0.2683,  0.2724, -0.0689,  0.0237,
        -0.1276, -0.0920, -0.2745,  0.1189,  0.0916,  0.3737,  0.0862,  0.3118],
       grad_fn=<CatBackward0>)

In [13]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [14]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3
                                   )

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [15]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [16]:
print(projection_list[0])

[0, tensor([[1.0529, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0529, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0529,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0529, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0529, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0529]]), 0, tensor([[1.2510, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.2510, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2510,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.2510, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2510, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2510]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [18]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.0016,  0.0798,  0.1991, -0.2646,  0.1492, -0.1215,  0.2237, -0.1344,
        -0.2093,  0.3116, -0.0348, -0.1772, -0.1844, -0.0705, -0.2113,  0.2130,
        -0.1988,  0.1714,  0.1038, -0.1824,  0.1971,  0.2693, -0.1247, -0.0079,
         0.0953,  0.2047, -0.0031, -0.2782, -0.1369, -0.0619,  0.2180,  0.1150,
         0.3067,  0.1449, -0.0419,  0.2904, -0.0664,  0.4803,  0.1630,  0.1145])


tensor([-0.0033,  0.1597,  0.3982, -0.5293,  0.2984, -0.2430,  0.4475, -0.2687,
        -0.4186,  0.6232, -0.0696, -0.3543, -0.3689, -0.1409, -0.4226,  0.4259,
        -0.3976,  0.3429,  0.2077, -0.3648,  0.3941,  0.5387, -0.2494, -0.0157,
         0.1906,  0.4094, -0.0061, -0.5564, -0.2738, -0.1238,  0.4359,  0.2299,
         0.6134,  0.2899, -0.0839,  0.5808, -0.1329,  0.9607,  0.3261,  0.2290],
       grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/2000 [00:02<1:14:26,  2.23s/it]

Training_loss 47.50122


  0%|          | 2/2000 [00:04<1:05:20,  1.96s/it]

Training_loss 47.47722


  0%|          | 3/2000 [00:05<58:49,  1.77s/it]  

...

Training_loss 20.30001


100%|█████████▉| 1999/2000 [44:59<00:01,  1.65s/it]

Training_loss 20.29303


100%|██████████| 2000/2000 [45:00<00:00,  1.35s/it]

Training_loss 20.24063


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-0.0460,  1.1812,  0.4610,  0.4469,  0.2511, -0.2240,  0.1706,  0.3388,
        -0.6996,  0.2031,  0.8270, -0.9598, -0.0157,  0.6698, -0.2592,  0.6487,
         0.8470,  0.0588,  1.2591, -0.5529,  0.2962, -0.2062, -0.2245, -0.2269,
         0.0198,  0.7513,  0.3471, -0.3338,  0.7348, -0.5020,  0.3347, -0.0425,
        -0.2992, -0.1321,  0.2386,  0.4576,  2.3006,  2.1637,  2.1098,  1.4697],
       grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

1
3


In [23]:
parameters_to_vector(models[0].parameters())

tensor([-0.1450,  0.1424,  0.0127,  0.0371, -0.0330, -0.2232, -0.2274, -0.1364,
         0.2306,  0.5960, -1.3876, -0.0987,  0.0926,  0.2494, -0.0292,  0.4427,
        -0.3820, -0.5574, -0.1521, -0.8897,  0.8261, -0.1268,  0.0452, -0.7876,
         0.0793,  1.2068, -0.3966, -1.2764,  0.4059, -0.2673,  0.4005,  0.1259,
        -0.1594,  0.1021,  0.9017,  0.9445,  0.5037,  1.8917,  2.2124,  2.0141],
       grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 tensor([[1.0529, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0529, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0529,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 1.0529, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0529, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0529]]),
 0,
 tensor([[1.2510, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.2510, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.2510,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 1.2510, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2510, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2510]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
projected_weights[0]

[0,
 tensor([ 0.1243, -0.1060, -0.2898,  0.3380,  0.2483,  0.0919,  0.2717,  0.1301,
         -0.3367, -0.0290,  0.3086,  0.1434,  0.3225, -0.0740,  0.1424,  0.2804,
         -0.2284, -0.3479,  0.1432,  0.2785,  0.2804,  0.1387, -0.0847, -0.1907,
         -0.3337, -0.1719,  0.3415, -0.2128, -0.0271,  0.3491, -0.3455, -0.1790,
          0.0926,  0.1035,  0.2593, -0.0221,  0.1437,  0.0775,  0.2320, -0.1122]),
 0,
 tensor([ 0.2555,  0.1692,  0.0845,  0.1881, -0.1884,  0.1152, -0.3656, -0.1480,
          0.2105, -0.0706, -0.2295,  0.1474, -0.2375,  0.0866,  0.0462,  0.0965,
          0.0970, -0.3092,  0.3336,  0.0669,  0.3328, -0.3561, -0.0212,  0.1061,
         -0.2490, -0.3698,  0.0536,  0.2303,  0.3627,  0.3105, -0.0190,  0.3036,
          0.3711, -0.1756,  0.3514, -0.2361, -0.3895,  0.0123,  0.1991,  0.4181]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
np.save( 'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
#np.save('relative_error_dfedu' + str(eta).replace('.', '_'), total_rel_error)